In [1]:
from symmer.projection import (
    S3_projection, QubitTapering, CS_VQE_LW, CS_VQE,
    StabilizerIdentification, ObservableBiasing, stabilizer_walk
)
from symmer.symplectic import *
from symmer.utils import exact_gs_energy
from symmer.utils import gf2_gaus_elim, gf2_basis_for_gf2_rref
from symmer.chem import geometry_from_pubchem, MoleculeBuilder
from matplotlib import pyplot as plt
from functools import reduce
import json

2022-05-30 15:43:31.579805: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-30 15:43:31.618449: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-30 15:43:31.621678: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
/home/tim/anaconda3/lib/python3.7/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
#mol_data = molecule_geometries[speciesname]
#atoms  = mol_data['atoms']
#coords = mol_data['coords']
geometry = [('Al', (0,0,0))] #list(zip(atoms, coords))
molecule = MoleculeBuilder(geometry=geometry, charge=+3, basis='STO-3G', spin=0, run_fci=True, print_info=True)

# taper the Hamiltonian
taper_hamiltonian = QubitTapering(molecule.H_q)
hf_array = molecule.H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
hf_tapered = taper_hamiltonian.tapered_ref_state

# initiate stabilizer identification classes
Hc_stabilizers = StabilizerIdentification(ham_tap)
CC_stabilizers = StabilizerIdentification(ucc_tap)
HL_gap = np.where(hf_tapered==0)[0][0]-.5
ham_biasing = ObservableBiasing(base_operator=ham_tap, HOMO_LUMO_gap=HL_gap)

# build CS-VQE model
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap, noncontextual_form='diag')


Molecule geometry:
Al	0	0	0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -237.25653909799263
MP2 energy:  -237.2633959975336
CCSD energy: -237.26342595760497
FCI energy:  -237.26355057978995


Number of qubits: 18


In [3]:
gs_nrg, gs_vec = exact_gs_energy(ham_tap.to_sparse_matrix)

In [4]:
gs_nrg - molecule.fci_energy

-1.2060418074637482

In [10]:
from itertools import product
for comb in product([+1, -1], repeat=4):
    ham_test = taper_hamiltonian.taper_it(sector=comb)
    gs_nrg, gs_vec = exact_gs_energy(ham_test.to_sparse_matrix)
    print(f'In sector {comb}, error = {gs_nrg - molecule.fci_energy}')

In sector (1, 1, 1, 1), error = -1.661622520301961
In sector (1, 1, 1, -1), error = -1.625669448635989
In sector (1, 1, -1, 1), error = -1.6256694486377796
In sector (1, 1, -1, -1), error = -1.2060418074627535
In sector (1, -1, 1, 1), error = -1.4407349197769292
In sector (1, -1, 1, -1), error = -1.4233963497653122
In sector (1, -1, -1, 1), error = -1.4233963497595994
In sector (1, -1, -1, -1), error = -1.4407349197822725
In sector (-1, 1, 1, 1), error = -1.4407349197784356
In sector (-1, 1, 1, -1), error = -1.4233963497590594
In sector (-1, 1, -1, 1), error = -1.4233963497584625
In sector (-1, 1, -1, -1), error = -1.4407349197822157
In sector (-1, -1, 1, 1), error = -1.2060418074635209
In sector (-1, -1, 1, -1), error = -1.6256694486366996
In sector (-1, -1, -1, 1), error = -1.625669448638348
In sector (-1, -1, -1, -1), error = -1.2060418074641746


In [27]:
ham_test = taper_hamiltonian.taper_it(sector=[0.5,0.5,0.5,0.5])
ham_test.n_terms

2092

In [ ]:
gs_nrg, gs_vec = exact_gs_energy(ham_test.to_sparse_matrix)

In [ ]:
gs_nrg - molecule.fci_energy

In [26]:
CC_op = molecule.H_q
ref_state = QuantumState([hf_array])

psi = CC_op * ref_state

for row in psi.state_op.symp_matrix:
    taper_hamiltonian.stabilizers.update_sector(hf_array)
    print(taper_hamiltonian.symmetry_generators.coeff_vec)

[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1  1  1]
[-1 -1

In [5]:
taper_hamiltonian.symmetry_generators

-1 ZIZIIZIZIZZIIZIZIZ 
-1 IZIZIZIZIZIZIZIZIZ 
 1 IIIIZZIIIIIIIIIIZZ 
 1 IIIIIIZZZZIIZZZZII

In [6]:
print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand may be rotated via a sequence of Clifford pi/2 rotations\n')
print(taper_hamiltonian.stabilizers.stabilizer_rotations)
#print('\nonto the single-qubit Pauli operators\n')
#print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
print(f'\nThe symetry sector corresponding with the reference {hf_array} is {taper_hamiltonian.stabilizers.coeff_vec}')
print(f'The tapered Hartree-Fock state is', QuantumState([hf_tapered]))

We are able to taper 3 qubits from the Hamiltonian.

The symmetry generators are

 1 ZIZIIZIZIZZIIZIZIZ 
 1 IZIZIZIZIZIZIZIZIZ 
 1 IIIIZZZZZZIIZZZZZZ

and may be rotated via a sequence of Clifford pi/2 rotations

[(1.000+0.000j YIZIIZIZIZZIIZIZIZ, None), (1.000+0.000j IYIZIZIZIZIZIZIZIZ, None), (1.000+0.000j IIIIYZZZZZIIZZZZZZ, None)]

The symetry sector corresponding with the reference [1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0] is [1 1 1]
The tapered Hartree-Fock state is  1.0000000000 |111111111000000>
